## Projeto 2 Ciência dos Dados 2020.1
___
- Daniel Gurgel Terra
- Gabriel Formario
___

### Dada as características biológica dos cogumelos é possível um machine learning determinar se eles são venenosos ou comestíveis ?

Nesse arquivo de notebook, construímos as funções que serão utilizadas nos outros notebooks para que fique mais fácil de entender os diferentes processos de classificação. Dessa maneira, os notebooks ficaram mais limpos e mais organizados.
Para que isso fosse possível, usamos a biblioteca "import_ipynb" (https://pypi.org/project/import-ipynb/).

## Importando bibliotecas do scikit-learn

Scikit-learn é uma biblioteca enorme de machine learnings que utiliza como linguagem o python, é super acessível

In [1]:
# O "train_test_split", usado como "separar_treino_teste", separa grupos de treino e teste
from sklearn.model_selection import train_test_split as separar_treino_teste
# Árvore de decisões
from sklearn.tree import DecisionTreeClassifier as DecisionTree
# Regressão Logística
from sklearn.linear_model import LogisticRegression
# Bernoulli Naive Bayes
from sklearn.naive_bayes import BernoulliNB
# Gaussiana Naive Bayes
from sklearn.naive_bayes import GaussianNB
# Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB

In [2]:
# Constrói gráficos
import matplotlib.pyplot as plt
# Constrói tabelas
import pandas as pd
# Realiza operações matemáticas
import numpy as np

In [3]:
#
#from sklearn.metrics import *

## Data

Os dados foram extraídos em csv ("coma separated values" ou "valores separados por vírgula"), são valores como em uma tabela de excel, mas todos separados por vírgula em um arquivo de texto, assim ele é muito leve e muito útil para programação

In [4]:
data = pd.read_csv("mushrooms.csv")

## Funções

* Json

Esse arquivo possui...

In [5]:
# lê arquivos no formato ".json"
import json
# abrindo o arquivo (não esquecer de por o "encoding=utf-8" pois senão não lê letras brasileiras: "ç", "~", "^"...)
with open('Renomeador.json', encoding="utf-8") as json_file:
    # guarda os valores do dicionário json numa variável
    renomeador = json.load(json_file)

* Limpeza dos dados 

Essa função coleta os novos nomes...

In [6]:
def Limpeza(data):
    # contador, para navegar entre as chaves da tabela a ser limpa
    i = 0
    # para cada chave e valor do dicionario
    for chave,valor in renomeador.items():
        # renomeia a coluna pela chave do dicionário
        data.rename(columns={data.columns[i]:chave}, inplace=True)
        # renomeia os valores na tabela
        data[chave].replace(valor, inplace=True)
        # vai para a próxima coluna da tabela
        i += 1

* Gráfico para cada característica do cogumelo no dataset

In [7]:
# Dataset passa pela limpeza para que a função abaixo funcione
Limpeza(data)

In [8]:
def Grafico_por_caracteristica(intervalo,data):
    # coleta o nome das colunas dadas pelo dicionário
    colunas = list(renomeador.keys())
    # para a quantidade de gráficos desejada
    for i in intervalo:
        # ajusta o tamanho dos gráficos
        plt.figure(figsize=(10,5))
        # separa em venenoso e comestível
        Venenoso = data[data['Periculosidade']=='venenoso'][colunas[i]].value_counts(True)*100
        Comestivel = data[data['Periculosidade']=='comestível'][colunas[i]].value_counts(True)*100
        # apresenta o gráfico de comestível para cada característica
        plt.bar( np.arange(len(Comestivel)), Comestivel.values.tolist(), width=.25, color="g")
        # apresenta o gráfico de venenoso para cada característica, 0.25 unidades adiante
        plt.bar( np.arange(len(Venenoso)) + [.25]*len(Venenoso), Venenoso.values.tolist(), width=.25, color="r")
        # coloca o nome das características (como são as mesmas para os dois, podia ter sido feito para comestível)
        plt.xticks(np.arange(len(Venenoso)) + len(Venenoso)*[0.125],Venenoso.axes[0].tolist())
        # nome do eixo vertical, horizontal e título
        plt.ylabel("Percentual"); plt.xlabel("Características"); plt.title(colunas[i])
        # legenda com as cores para comestível e venenoso
        plt.legend(['Comestível','Venenoso'],title="Periculosidade")
    # apresenta o gráfico
    plt.show()

* Encoder

Processo que transforma as variavéis qualitativas em quantitativas de acordo com a respectiva característica

Técnica encontrada em : https://www.kaggle.com/nirajvermafcb/comparing-various-ml-models-roc-curve-comparison

In [9]:
print("Qualitativo : ", data['Ligação Branqueal'].unique())

Qualitativo :  ['Livre' 'Ligado']


In [10]:

from sklearn.preprocessing import LabelEncoder

labelencoder=LabelEncoder()

for col in data.columns:
    
    data[col] = labelencoder.fit_transform(data[col])

# o dataset agora possui uma numeração para cada característica
print("Quantitativo : ", data['Ligação Branqueal'].unique())

Quantitativo :  [1 0]


* Determinando os dados para o classificador

In [11]:
# os dados a serem computados, sem as respostas
tabela = data.drop(columns=["Periculosidade"])
# coluna-objetivo, a coluna a qual deseja-se prever
objetivo = data["Periculosidade"]
# tamanho do teste em relação ao total
tamanho_teste = 0.8
# separando em treino e teste
X_treino, X_teste, Y_treino, Y_teste = separar_treino_teste(objetivo, tabela, test_size=tamanho_teste)

* Funcao de previsao de modelos utilizadas no Machine Learning

In [12]:
def Previsão(tipo,logistica=False):
    
    if logistica:
        
        model = tipo(solver='lbfgs',multi_class='auto',max_iter=1000)
        
    else:
        
        model = tipo()
    
    model.fit(X_treino,Y_treino);
    
    model.score(X_teste,Y_teste);
    
    Y_predict = model.predict(X_teste);
    
    acertos=0
    
    for i in range(len(X_teste)):
        
        if Y_predict[i]==Y_teste.iloc[i]:
            
            acertos+=1;
            
    erros = len(X_teste)-acertos
    
    return round(acertos/len(X_teste)*100,2), round(erros/len(X_teste)*100,2)